In [1]:
import pandas as pd
import numpy as np

### First part: data cleaning and sampling
Randomly sampling 1% of loans data from 2008 Q1 to 2009 Q4 
the sampled data need to meet the following criteria: 
* Single family home
* 30-year fixed rate
* FICO score between 550 and 850

In [162]:
def importdata(path):
    #import data and creat column names
    data=pd.read_csv(path,sep="|",header=None,low_memory=False)
    colnames=("CREDIT_SCORE","FIRST_PAYMENT_DATE","FIRST_TIME_HOMEBUYER_FLAG",
          "MATURITY DATE","MSA","MI","NUMBER OF UNITS","OCCUPANCY STATUS",
          "CLTV","ORIGINAL DTI","ORIGINAL UPB","ORIGINAL LTV","ORIGINAL INTEREST RATE",
          "CHANNEL","PPM FLAG","PRODUCT TYPE","PROPERTY STATE","PROPERTY TYPE","POSTAL CODE",
          "LOAN SEQUENCE NUMBER","LOAN PURPOSE","ORIGINAL LOAN TERM","NUMBER OF BORROWERS",
          "SELLER NAME","SERVICER NAME","SUPER CONFORMING FLAG")
    colnames=[item.replace(" ","_")for item in colnames]
    data.columns=colnames
    #find records meet the criteria
    data_wanted=data[(data["CREDIT_SCORE"]>=550)&(data["CREDIT_SCORE"]<=850)&(data["PROPERTY_TYPE"]=="SF")&(data["ORIGINAL_LOAN_TERM"]==360)]
    #sample 1% of the data
    sampled_data=data_wanted.sample(frac=0.01)
    return sampled_data

In [163]:
#input data with the function defined above
#the data is downloaded by quarters so we have to do it repeatedly
sampledata0801=importdata("historical_data1_Q12008.txt")
sampledata0802=importdata("historical_data1_Q22008.txt")
sampledata0803=importdata("historical_data1_Q32008.txt")
sampledata0804=importdata("historical_data1_Q42008.txt")
sampledata0901=importdata("historical_data1_Q12009.txt")
sampledata0902=importdata("historical_data1_Q22009.txt")
sampledata0903=importdata("historical_data1_Q32009.txt")
sampledata0904=importdata("historical_data1_Q42009.txt")

In [164]:
#combine 8 sample data 
tsample=pd.concat([sampledata0801,sampledata0802,sampledata0803,sampledata0804,
           sampledata0901,sampledata0902,sampledata0903,sampledata0904], axis=0)
sample=tsample.reset_index(drop=True)

In [165]:
#output clean sample data into csv file
sample.to_csv("sampledata_20082009",index=False)

### Second part: testing some hypothesis with A/B testing
##### Hypothesis:
* After the financial meltdown, loans in general have high credit scores (FICO).
* After the financial meltdown, the portion of subprime loans (FICO < 660) decreased
substantially in the mortgage portfolio.
*  The quality of loans originated after meltdown is considerably higher, i.e., their
delinquency rate is much lower compared to those loans originated before the meltdown.



### First hypothesis : 

After the financial meltdown, loans in general have high credit scores (FICO)
* In order to check the hypothesis, we need to detect if the difference in the credict score before and after crisis is statistically significant
* we randomly select similar number of samples before and after crisis as control and experiment group and do an A/B testing with α=0.05
* if the difference is statistically significant, the hypothesis is true.


$$
SD={S}_{pool} \sqrt{\frac{1}{{N}_{cont}}+\frac{1}{{N}_{exp}}}
$$


$$
{S}_{pool}=\frac{{SS}_{cont}+{SS}_{exp}}{df}
$$

In [167]:
#first step: prepare data, we sample the experiment data to make sure the size of the population in two groups are comparable
control=sample[sample["FIRST_PAYMENT_DATE"]<200809]["CREDIT_SCORE"]
experiment=sample[sample["FIRST_PAYMENT_DATE"]>=200809].sample(frac=0.33)["CREDIT_SCORE"]
N_cont=len(control)
N_exp=len(experiment)
print('size of control group:',len(control))
print('size of experiment group:',len(experiment))

size of control group: 4507
size of experiment group: 4465


In [168]:
# since the sample size is large enough, we can assume it follows normal distribution
# assume the variance between two groups are similar
SS_cont=sum((control-control.mean())**2)
SS_exp=sum((experiment-experiment.mean())**2)
S_pool=((SS_cont+SS_exp)/(N_cont+N_exp-2))**0.5
SD=S_pool*(1/N_cont+1/N_exp)**0.5
E=SD*1.96
d_estimate=experiment.mean()-control.mean()
print("Confidence Interval for credit score: [{},{}]".format(round(d_estimate-E,4),round(d_estimate+E,4)))

Confidence Interval for credit score: [22.1651,26.1861]


##### Conclusion:

The Hypothesis that after the financial meltdown, loans in general have high credit scores is true. The confidence interval does not include 0, so we are confidenct at a 95% significant level that the difference in credict score before and after crisis is statistically significant.



### Second hypothesis : 

After the financial meltdown, the portion of subprime loans (FICO < 660) decreased substantially in the mortgage portfolio.
* In order to check the hypothesis, we need to detect if the difference in the subprime loans probability before and after crisis is statistically significant. In another word we regard before and after crisis as control and experiment group.
* we randomly select 100 records to calculate subprime loans probability before and after crisis. Repeat the process for 30 times to collect 30 average subprime loans probability in each group. Then do an A/B testing with α=0.05 based on the data we collected.
* if the difference is statistically significant, the hypothesis is true.

##### First step: Preparing data
   
Randomly select 100 records to calculate subprime loans probability before and after crisis. Repeat the process for 30 times.

In [169]:
data_beforeCrisis=sample[sample["FIRST_PAYMENT_DATE"]<200809]
data_afterCrisis=sample[sample["FIRST_PAYMENT_DATE"]>200809]

In [170]:
# sample around 100 data and calculate subprime number
def get_Subprime_pro(data,fraction):
    sample=data.sample(frac=fraction)
    N_subprime=len(sample[sample["CREDIT_SCORE"]<660])
    return N_subprime

In [171]:
# get 30 subprime loan number before crisis.
#in order to sample 100 loans, we set the fraction as 100/number of loan before crisis
Subprime_cont=[get_Subprime_pro(data_beforeCrisis,100/len(data_beforeCrisis)) for i in range(30)]
Loan_cont=[len(data_beforeCrisis.sample(frac=100/len(data_beforeCrisis))) for i in range(30)]
# get 30 subprime loan number after crisis
#in order to sample 100 loans, we set the fraction as 100/number of loan after crisis
Subprime_exp=[get_Subprime_pro(data_afterCrisis,100/len(data_afterCrisis)) for i in range(30)]
Loan_exp=[len(data_afterCrisis.sample(frac=100/len(data_afterCrisis))) for i in range(30)]

In [172]:
column_name=["Loan_cont","Subprime_cont","Loan_exp","Subprime_exp"]
df_Subprime = pd.DataFrame(list(zip(Loan_cont,Subprime_cont,Loan_exp,Subprime_exp)), columns =column_name)
df_Subprime.head()

,Loan_cont,Subprime_cont,Loan_exp,Subprime_exp
0,100,9,100,4
1,100,10,100,3
2,100,9,100,3
3,100,11,100,1
4,100,14,100,3


##### Second step: Calculate S_pool and p_pool
$$
{S}_{pool}= \sqrt{{p}_{pool}*(1-{p}_{pool})*(\frac{1}{{N}_{cont}}+\frac{1}{{N}_{exp}})}
$$


$$
{p}_{pool}=\frac{{C}_{cont}+{C}_{exp}}{{N}_{cont}+{N}_{exp}}
$$

In [173]:
Subprime_cont=df_Subprime['Subprime_cont'].sum()
Subprime_exp=df_Subprime['Subprime_exp'].sum()
Loan_cont=df_Subprime['Loan_cont'].sum()
Loan_exp=df_Subprime['Loan_exp'].sum()
p_pool=(Subprime_cont+Subprime_exp)/(Loan_cont+Loan_exp)
S_pool=(p_pool*(1-p_pool)*(1/Loan_cont+1/Loan_exp))**0.5

##### Third step: Calculate confidence interval

In [174]:
E=S_pool*1.96
d_estimate=Subprime_exp/Loan_exp-Subprime_cont/Loan_cont
print("Confidence Interval for subprime loans probability: [{},{}]".format(round(d_estimate-E,4),round(d_estimate+E,4)))

Confidence Interval for subprime loans probability: [-0.0774,-0.0533]


##### Conclusion:

The Hypothesis that after the financial meltdown, the portion of subprime loans (FICO < 660) decreased substantially in the mortgage portfolio is true. The confidence interval does not include 0, so we are confidenct at a 95% significant level that the difference in portion of subprime loans before and after crisis is statistically significant.



### Third hypothesis : 
The quality of loans originated after meltdown is considerably higher, i.e., their delinquency rate is much lower compared to those loans originated before the meltdown.

##### We need to define delinquency first:
* A loan has reached 90+ days in delinquency
* A loan has shown recovery amount as recovery usually comes after foreclosure and repossession by the bank (REO)
* A loan has been modified, i.e., interest rate reduction after negotiating with the bank.

Based on this definition we need to use performance data to derive a delinquency flag for each loan.
As older loans have longer performance histories and hence are more likely to encounter delinquency, we select performance data of 5 years for each loan.

#### first step: derive a delinquency flag for each loan
##### 1. find loan sequence number in sample dataset

In [4]:
#input a new 'sample' data 
sample_with_delinquency=pd.read_csv("sampledata_20082009")
# add a delinquency column in sample data and assign 'N'
sample_with_delinquency['delinquency']='N'
# find loan sequence number in sample dataset
Loan_id=sample["LOAN_SEQUENCE_NUMBER"].unique()

##### 2. load and clean the historical data


* write a function to load performance historical data in chunk.
* filter the historical data to find corresponding historical data of these loans in sample data, because the historical data contains all the historical data of loans that has not been paied off.
* filter the historical data within five years.

In [5]:
def read_big_data(path,year):
    mylist = []
    for chunk in  pd.read_csv(path, sep="|", chunksize=20000,
                         header=None,low_memory=False,
                         usecols=[0, 1,3, 7,18]):
        mylist.append(chunk)
    big_data = pd.concat(mylist, axis= 0)
    del mylist
    big_data.columns=["LOAN_SEQUENCE_NUMBER","MONTHLY_REPORTING_PERIOD","CURRENT_LOAN_DELINQUENCY_STATUS","MODIFICATION_FLAG","MAINTENANCE_AND_PRESERVATION_COSTS"]
    #filter records within 5 years
    filter_l=big_data["LOAN_SEQUENCE_NUMBER"].isin(Loan_id)
    filter_y=big_data["MONTHLY_REPORTING_PERIOD"]<=year
    history_data=big_data[filter_l&filter_y]
    return history_data

##### 3. label delinquency in sample data

In [17]:
#define a function to find delinquency loan and label it in 'sample' data
def label_delinquency(input_historydata,sample_with_delinquency):
    #find delinquency loan based on definition
    filter1=input_historydata['CURRENT_LOAN_DELINQUENCY_STATUS']==3
    filter2=input_historydata['MODIFICATION_FLAG']=='Y'
    filter3=input_historydata['MAINTENANCE_AND_PRESERVATION_COSTS']>0
    delinquency_id=input_historydata[filter1|filter2|filter3]['LOAN_SEQUENCE_NUMBER'].unique()
    #label the delinquency in 'sample' data
    sample_with_delinquency.loc[sample_with_delinquency["LOAN_SEQUENCE_NUMBER"].isin(delinquency_id),'delinquency']='Y'
    return sample_with_delinquency

In [ ]:
# use function defined above to load all historical data within 5 years of loans in sample

In [7]:
# use function defined above to load all historical data within 5 years of loans in sample
history_data08Q1=read_big_data('historical_data1_time_Q12008.txt',201304)
history_data08Q2=read_big_data('historical_data1_time_Q22008.txt',201307)
history_data08Q3=read_big_data('historical_data1_time_Q32008.txt',201310)
history_data08Q4=read_big_data('historical_data1_time_Q42008.txt',201401)

In [11]:
history_data09Q1=read_big_data('historical_data1_time_Q12009.txt',201404)
history_data09Q2=read_big_data('historical_data1_time_Q22009.txt',201407)
history_data09Q3=read_big_data('historical_data1_time_Q32009.txt',201410)
history_data09Q4=read_big_data('historical_data1_time_Q42009.txt',201501)

In [18]:
#use function defined above to label delinquency in sample data
sample_with_delinquency=label_delinquency(history_data08Q1,sample_with_delinquency)
sample_with_delinquency=label_delinquency(history_data08Q2,sample_with_delinquency)
sample_with_delinquency=label_delinquency(history_data08Q3,sample_with_delinquency)
sample_with_delinquency=label_delinquency(history_data08Q4,sample_with_delinquency)

In [20]:
sample_with_delinquency=label_delinquency(history_data09Q1,sample_with_delinquency)
sample_with_delinquency=label_delinquency(history_data09Q2,sample_with_delinquency)
sample_with_delinquency=label_delinquency(history_data09Q3,sample_with_delinquency)
sample_with_delinquency=label_delinquency(history_data09Q4,sample_with_delinquency)

In [26]:
sample_with_delinquency[["LOAN_SEQUENCE_NUMBER","FIRST_PAYMENT_DATE","delinquency"]].head()

,LOAN_SEQUENCE_NUMBER,FIRST_PAYMENT_DATE,delinquency
0,F108Q1003851,200803,N
1,F108Q1155511,200804,Y
2,F108Q1234811,200805,N
3,F108Q1092373,200804,Y
4,F108Q1137346,200803,N


#### second step: prepare data for A/B testing
* In order to check the hypothesis, we need to detect if the difference in the delinquency probability before and after crisis is statistically significant. In another word we regard before and after crisis as control and experiment group.
* we randomly select 200 records to calculate delinquency probability before and after crisis. Repeat the process for 30 times to collect 30 average delinquency probability in each group. Then do an A/B testing with α=0.05 based on the data we collected.
* if the difference is statistically significant, the hypothesis is true.

In [27]:
# define a function to sample around 200 data and calculate delinquency number
def get_delinquency_N(sample_with_delinquency,fraction):
    sample=sample_with_delinquency.sample(frac=fraction)
    N_delinquency=len(sample[sample["delinquency"]=='Y'])
    return N_delinquency

In [28]:
# split data with delinquency into before and after crisis
data_wd_beforeCrisis=sample_with_delinquency[sample_with_delinquency["FIRST_PAYMENT_DATE"]<200809]
data_wd_afterCrisis=sample_with_delinquency[sample_with_delinquency["FIRST_PAYMENT_DATE"]>200809]

In [29]:
# get 30 delinquency number before and after crisis.
# in order to sample 200 loans, we set the fraction as 200/number of loan before and after crisis.
Delinquency_cont=[get_delinquency_N(data_wd_beforeCrisis,200/len(data_wd_beforeCrisis)) for i in range(30)]
Loan_cont=[len(data_wd_beforeCrisis.sample(frac=200/len(data_wd_beforeCrisis))) for i in range(30)]

Delinquency_exp=[get_delinquency_N(data_wd_afterCrisis,200/len(data_wd_afterCrisis)) for i in range(30)]
Loan_exp=[len(data_wd_afterCrisis.sample(frac=200/len(data_wd_afterCrisis))) for i in range(30)]

In [31]:
column_name=["Loan_cont","Delinquency_cont","Loan_exp","Delinquency_exp"]
df_Delinquency = pd.DataFrame(list(zip(Loan_cont,Delinquency_cont,Loan_exp,Delinquency_exp)), columns =column_name)
df_Delinquency.head()

,Loan_cont,Delinquency_cont,Loan_exp,Delinquency_exp
0,200,6,200,1
1,200,6,200,3
2,200,9,200,2
3,200,1,200,2
4,200,9,200,3


##### Third step: Calculate S_pool and p_pool
$$
{S}_{pool}= \sqrt{{p}_{pool}*(1-{p}_{pool})*(\frac{1}{{N}_{cont}}+\frac{1}{{N}_{exp}})}
$$


$$
{p}_{pool}=\frac{{C}_{cont}+{C}_{exp}}{{N}_{cont}+{N}_{exp}}
$$

In [33]:
Delinquency_cont=df_Delinquency['Delinquency_cont'].sum()
Delinquency_exp=df_Delinquency['Delinquency_exp'].sum()
Loan_cont=df_Delinquency['Loan_cont'].sum()
Loan_exp=df_Delinquency['Loan_exp'].sum()
p_pool=(Delinquency_cont+Delinquency_exp)/(Loan_cont+Loan_exp)
S_pool=(p_pool*(1-p_pool)*(1/Loan_cont+1/Loan_exp))**0.5

##### Forth step: Calculate confidence interval

In [34]:
E=S_pool*1.96
d_estimate=Delinquency_exp/Loan_exp-Delinquency_cont/Loan_cont
print("Confidence Interval for loan delinquency number: [{},{}]".format(round(d_estimate-E,4),round(d_estimate+E,4)))

Confidence Interval for loan delinquency number: [-0.032,-0.0213]


##### Conclusion:

The Hypothesis that the quality of loans originated after meltdown is considerably higher. In another word, their delinquency rate is much lower compared to those loans originated before the meltdown is true. The confidence interval does not include 0, so we are confidenct at a 95% significant level that the difference in delinquency rate before and after crisis is statistically significant.